# 第5章: 係り受け解析

```bash
$ cat data/neco.txt | cabocha -f1 > data/neko.txt.cabocha
```

## 40. 係り受け解析結果の読み込み（形態素）

In [1]:
class Morph:
    def __init__(self, nd):
        self.surface = nd['surface']
        self.base = nd['base']
        self.pos = nd['pos']
        self.pos1 = nd['pos1']

with open('../data/neko.txt.cabocha', encoding='utf-8') as f:
    l = f.read().split('EOS\n')
f.close()

In [2]:
ls = list(filter(lambda x: x != '',l))
txtArr=[]
#surface \t part-of-speech, sub-class1, sub-class2, sub-class3, inflection, conjugation, base, reading, pronunciation
for line in ls:
    arr=[]
    for w in line.split('\n'):
        if w == '':continue
        elif w[0] == '*':continue
        surf,feat = w.split('\t')
        feat=feat.split(',')
        arr.append(
            Morph({
                    'surface': surf,
                    'base'   : feat[6],
                    'pos'    : feat[0],
                    'pos1'   : feat[1]
            })
        )
        
    txtArr.append(arr)

In [3]:
[x.surface for x in txtArr[6]]

['しかも',
 'あと',
 'で',
 '聞く',
 'と',
 'それ',
 'は',
 '書生',
 'という',
 '人間',
 '中',
 'で',
 '一番',
 '獰悪',
 'な',
 '種族',
 'で',
 'あっ',
 'た',
 'そう',
 'だ',
 '。']

## 41. 係り受け解析結果の読み込み（文節・係り受け）

In [4]:
class Chunk:
    def __init__(self, morphs, dst):
        self.morphs = morphs
        self.dst = dst
        self.srcs = []

In [5]:
result=[]
for line in ls:
    lsChunk=[]
    #print(line)
    for w in line.split('\n'):
        if w == '':continue
        elif w[0] == '*': # chunk → 毎回文節の最初
            dst=w.split(' ')[2].rstrip('D')
            # 　新規Chunkオブジェクト作成、チャンクリスト末尾に入れる
            lsChunk.append(Chunk([],int(dst)))
        else: # 単語
            surf,feat = w.split('\t')
            feat=feat.split(',')
            # 　lsChunkの最後のChunkオブジェクト（この単語を含む文節の最初で作成される）
            # 　取り出したlsChunkのmorphsリストに入れる
            lsChunk[-1].morphs.append(
                Morph({
                        'surface': surf,
                        'base'   : feat[6],
                        'pos'    : feat[0],
                        'pos1'   : feat[1]
                })
            )
            
    for i,c in enumerate(lsChunk):
        # dist番目のチャンクのsrcsリストに各チャンクのindexを入れる
        lsChunk[lsChunk[i].dst].srcs.append(i)
    
    result.append(lsChunk)
    
for ch in result[7]:
    print(f"{''.join([m.surface for m in ch.morphs])} (-> {''.join([w.surface for w in result[7][ch.dst].morphs])})")

この (-> 書生というのは)
書生というのは (-> 話である。)
時々 (-> 捕えて)
我々を (-> 捕えて)
捕えて (-> 煮て)
煮て (-> 食うという)
食うという (-> 話である。)
話である。 (-> 話である。)


## 42. 係り元と係り先の文節の表示

In [6]:
for chList in result[0:10]:
    for ch in chList:
        print(''.join([m.surface for m in list(filter(lambda x: x.pos != '記号', ch.morphs))]),
              ''.join([w.surface for w in list(filter(lambda x: x.pos != '記号', chList[ch.dst].morphs))]),
             sep='\t')

一	一
	猫である
吾輩は	猫である
猫である	猫である
名前は	無い
まだ	無い
無い	無い
どこで	生れたか
生れたか	つかぬ
とんと	つかぬ
見当が	つかぬ
つかぬ	つかぬ
何でも	薄暗い
薄暗い	所で
じめじめした	所で
所で	泣いて
ニャーニャー	泣いて
泣いて	記憶している
いた事だけは	記憶している
記憶している	記憶している
吾輩は	見た
ここで	始めて
始めて	人間という
人間という	ものを
ものを	見た
見た	見た
しかも	種族であったそうだ
あとで	聞くと
聞くと	種族であったそうだ
それは	種族であったそうだ
書生という	人間中で
人間中で	種族であったそうだ
一番	獰悪な
獰悪な	種族であったそうだ
種族であったそうだ	種族であったそうだ
この	書生というのは
書生というのは	話である
時々	捕えて
我々を	捕えて
捕えて	煮て
煮て	食うという
食うという	話である
話である	話である
しかし	思わなかった
その	当時は
当時は	なかったから
何という	考も
考も	なかったから
なかったから	思わなかった
別段	恐し
恐し	思わなかった
いとも	思わなかった
思わなかった	思わなかった
ただ	載せられて
彼の	掌に
掌に	載せられて
載せられて	持ち上げられた
スーと	持ち上げられた
持ち上げられた	時
時	フワフワした
何だか	フワフワした
フワフワした	感じが
感じが	あったばかりである
あったばかりである	あったばかりである


## 43. 名詞を含む文節が動詞を含む文節に係るものを抽出

In [7]:
for chList in result[0:100]:
    for ch in chList:
        txt=''.join([m.surface for m in list(filter(lambda x: x.pos != '記号', ch.morphs))])
        dstTxt=''.join([w.surface for w in list(filter(lambda x: x.pos != '記号', chList[ch.dst].morphs))])
        posList=[m.pos for m in list(filter(lambda x: x.pos != '記号', ch.morphs))]
        posDstList=[m.pos for m in list(filter(lambda x: x.pos != '記号', chList[ch.dst].morphs))]
        
        if '名詞' in posList and '動詞' in posDstList:
            print(txt, dstTxt,sep='\t')

どこで	生れたか
見当が	つかぬ
所で	泣いて
ニャーニャー	泣いて
いた事だけは	記憶している
記憶している	記憶している
吾輩は	見た
ここで	始めて
ものを	見た
あとで	聞くと
我々を	捕えて
掌に	載せられて
スーと	持ち上げられた
時	フワフワした
感じが	あったばかりである
上で	落ちついて
顔を	見たのが
ものの	見始であろう
見始であろう	見始であろう
ものだと	思った
感じが	残っている
今でも	残っている
第一毛をもって	装飾されべきはずの
顔が	つるつるして
その後	逢ったが
猫にも	逢ったが
一度も	出会わした
真中が	突起している
突起している	突起している
中から	吹く
ぷうぷうと煙を	吹く
咽せぽくて	弱った
人間の	飲む
事は	知った
頃	知った
裏で	坐って
心持に	坐って
速力で	運転し始めた
運転し始めた	運転し始めた
書生が	動くのか
動くのか	動くのか
自分だけが	動くのか
動くのか	分らないが
眼が	廻る
胸が	悪くなる
音が	して
眼から	出た
火が	出た
それまでは	記憶しているが
記憶しているが	分らない
あとは	分らない
事やら	分らない
気が	付いて
書生は	いない
たくさん	おった
兄弟が	見えぬ
一疋も	見えぬ
母親さえ	隠してしまった
姿を	隠してしまった
所とは	違って
眼を	明いていられぬくらいだ
吾輩は	棄てられたのである
上から	棄てられたのである
急に	棄てられたのである
中へ	棄てられたのである
棄てられたのである	棄てられたのである
思いで	這い出すと
笹原を	這い出すと
向うに	ある
池が	ある
吾輩は	見た
前に	坐って
分別も	出ない
書生がまた	来てくれるかと
迎に	来てくれるかと
試みに	やって
誰も	来ない
上を	渡って
風が	渡って
日が	かかる
暮れ	かかる
腹が	減って来た
非常に	減って来た
声が	出ない
何でも	廻り始めた
食物の	ある
所まで	ある
決心を	して
池を	廻り始めた
左りに	廻り始めた
そこを	我慢して
我慢して	這って行くと
無理やりに	這って行くと
事で	出た
所へ	出た
ここへ	這入ったら
竹垣の	崩れた
穴から	もぐり込んだ
邸内に	もぐり込んだ
もので	餓死したかも知れんのである
竹垣が	破れていなかったなら
吾輩は	餓死したかも知れんのである

## 44. 係り受け木の可視化

In [8]:
import pydot

coms=[]
for chList in [result[7]]:
    for ch in chList:
        txt=''.join([m.surface for m in list(filter(lambda x: x.pos != '記号', ch.morphs))])
        dstTxt=''.join([w.surface for w in list(filter(lambda x: x.pos != '記号', chList[ch.dst].morphs))])
        if txt != '' and dstTxt != '':
            coms.append((txt,dstTxt))

g=pydot.graph_from_edges(coms)
g.write_png('../data/44.png', prog='dot')

True

## 45. 動詞の格パターンの抽出

In [9]:
n,t='\n','\t'
res=[]
with open('../data/45.txt', mode='w') as f:
    for chList in result:
        for ch in chList:
            ils=[idx for idx, x in enumerate([m.pos for m in list(filter(lambda x: x.pos != '記号', ch.morphs))]) if x == '動詞']
            if len(ils)==0:continue
            for i in ils:
                postPosList=[m.base for m in list(filter(lambda x: x.pos != '記号', sum([chList[idx].morphs for idx in ch.srcs],[]))) if m.pos == '助詞']
                f.write(f"{ch.morphs[i].base}{t}{' '.join(postPosList)}{n}")
f.close()      
        

In [10]:
!cat ../data/45.txt | sort | uniq -c | sort -nr 

   2920 する	が
   1968 つく	か が
   1122 する	
   1014 する	が と で
    654 云う	に
    496 いる	は て を て
    436 られる	に
    357 られる	て と
    328 する	まで は
    316 見る	の
    255 云う	と は
    250 思う	と
    250 かける	
    244 「	
    241 ある	まで
    201 する	が は へ と を
    183 たらす	に を て
    143 する	まで が を
    135 出る	も
    128 見せる	て
    128 来る	が
    125 ある	が かい
    123 思う	と て
    117 いる	が でも て
    114 云う	
    113 云う	て は と
    112 見る	は を
    108 出る	の に
    106 行く	へ
    105 する	から て て を
     98 出来る	て
     96 聞く	で
     92 する	て だけ は て
     84 あるく	へ へ と て
     80 なる	ものの に
     80 つづける	を
     77 いる	と て に が に て
     76 見る	
     74 聞く	で を
     73 来る	が に
     72 知る	は
     70 行く	て に
     67 なる	と に かも
     67 ひろげる	で を
     67 あきれる	
     63 食う	て
     63 出る	て も が
     56 聞く	
     53 出来る	で が
     51 見る	は て て
     50 しいる	じゃ と
     49 分る	か
     47 生れる	で
     47 云う	から と
     46 く	と
     45 たつ	なんか
     44 見える	が も
     44 知る	に を
     44 くる	は て から て
     43 来る	が に て
     43 持つ	を
     43 出る	て と で へ
     42 行く	は に は
     42 云う	とか
     41 這入る	へ


In [11]:
!cat ../data/45.txt | sort | grep -E '^する|^見る|^与える' | uniq -c | sort -nr

   1098 する	が
    749 する	が て
    400 する	
    353 する	が と で
    316 見る	の
    153 する	は は は と
    124 する	まで は
    112 見る	は を
     96 する	まで
     76 見る	
     56 する	が は へ と を
     52 する	まで が を
     51 見る	は て て
     44 する	に でも
     38 する	だって は
     37 する	から て て を
     31 する	で でも を
     28 する	て だけ は て
     26 見る	ので が
     21 見る	から
     20 する	として
     20 する	で は から
     18 する	か て も に は を
     12 する	は ながら を
     12 する	と て たり を て
     11 する	くらい に に
     10 する	から に も て を
     10 する	も くらい
     10 する	の に まで を
      9 見る	より で を
      9 見る	か ね と を
      8 見る	は て で を て
      8 する	をもって
      8 する	て は に かも
      7 する	によって と は
      7 する	から って を
      7 する	て だけ を に に
      7 する	は に ながら を
      7 する	の て と は で て の
      6 見る	に って を
      6 見る	て まで
      6 する	から さえ
      6 する	が によって
      6 する	に で を から に
      5 与える	ば を
      5 する	について を
      5 する	たって ねえ
      5 する	かい と さえ を
      5 する	から な と は の に
      5 する	て て だけ に よ と
      4 見る	たり て が を
      4 する	じゃ だけ は に
      4 する	は ほど は と を に
      4 する	と は が て な と は

## 46. 動詞の格フレーム情報の抽出

In [12]:
n,t='\n','\t'
with open('../data/46.txt', mode='w') as f:
    for chList in result:
        for ch in chList:
            ils=[idx for idx, x in enumerate([m.pos for m in list(filter(lambda x: x.pos != '記号', ch.morphs))]) if x == '動詞']
            if len(ils)==0:continue
            for i in ils:
                postPosList=[m.base for m in list(filter(lambda x: x.pos != '記号', sum([chList[idx].morphs for idx in ch.srcs],[]))) if m.pos == '助詞']
                l1=list(filter(lambda m: '助詞' in [x.pos for x in m], [chList[idx].morphs for idx in ch.srcs]))
                txtLs=[''.join([x.surface for x in hoge]) for hoge in l1]
                f.write(f"{ch.morphs[i].base}{t}{' '.join(postPosList)}{t}{' '.join(txtLs)}{n}")
f.close()

## 47. 機能動詞構文のマイニング

In [13]:
n,t='\n','\t'
with open('../data/47.txt', mode='w') as f:
    for chList in result:
        for i,ch in enumerate(chList):
            ils=[idx for idx, x in enumerate([m.pos1 for m in ch.morphs]) if x == 'サ変接続']
            if (len(ils)==0 or len(ch.morphs)<=ils[0]+1 or ch.morphs[ils[0]+1].base != 'を') \
                or ('動詞' not in [x.pos for x in list(filter(lambda x: x.pos != '記号',chList[ch.dst].morphs))]) :continue    
            sams=list(filter(lambda x: x.pos != '記号' and (x.pos1 =='サ変接続' or x.base=='を'), list(filter(lambda x: x.pos != '記号',ch.morphs))))
            vbms=list(filter(lambda x: x.pos == '動詞', list(filter(lambda x: x.pos != '記号',chList[ch.dst].morphs))))
            saSrcsMos=[list(filter(lambda x: x.pos != '記号',chList[idx].morphs)) for idx in ch.srcs]
            vbSrcsMos=list(filter(lambda x: x not in sams+vbms,[list(filter(lambda x: x.pos != '記号',chList[idx].morphs)) for idx in list(filter(lambda x: x not in [i,i+1],chList[ch.dst].srcs))]))
            postPosList=[x.surface for x in sum([list(filter(lambda x: x.pos == '助詞',l)) for l in saSrcsMos+vbSrcsMos],[])]
            srcsSurfList=[''.join([x.surface for x in ms]) for ms in saSrcsMos+vbSrcsMos]

            f.write(f"{''.join([x.surface for x in sams])+vbms[0].base}{t}{' '.join(postPosList)}{t}{' '.join(srcsSurfList)}{n}")
        
    f.close()

In [14]:
!cut -f 1 ../data/47.txt | sort | uniq -c | sort -nr 

     30 返事をする
     21 挨拶をする
     17 話をする
     16 真似をする
     13 喧嘩をする
      9 運動をする
      9 質問をする
      6 話を聞く
      6 注意をする
      6 昼寝をする
      6 問答をする
      5 質問をかける
      5 講義をする
      5 相談をする
      5 病気をする
      5 戦争をする
      4 降参をする
      4 辞儀をする
      4 講釈をする
      4 演説をする
      4 欠伸をする
      4 散歩をする
      4 放蕩をする
      4 休養を要する
      4 いたずらをする
      3 遠慮をする
      3 逆上をする
      3 議論をする
      3 談話を聞く
      3 談話をする
      3 行水を使う
      3 行動をとる
      3 落着を告げる
      3 自殺をする
      3 経験をする
      3 深入りをする
      3 活躍を試みる
      3 決心をする
      3 批評を願う
      3 往来をあるく
      3 安心を得る
      3 学問をする
      3 噂をする
      3 告訴をかく
      3 同情を表する
      3 同情を表す
      3 化粧をする
      3 写生をする
      3 体操をする
      3 交際をする
      2 逆戻りをする
      2 迫害を加える
      2 辞職をする
      2 賭をする
      2 談判を聞く
      2 談判をする
      2 説教をやる
      2 話をつづける
      2 註釈をつける
      2 観察を怠る
      2 覚悟をする
      2 見物をする
      2 著述をする
      2 苦心をする
      2 自炊をする
      2 自慢をする
      2 結婚を申し込む
      2 結婚をやる
      2 約束をする
      2 研究をやる
      2 注

In [15]:
!cut -f 1,2 ../data/47.txt | sort | uniq -c | sort -nr 

      8 運動をする	
      7 返事をする	は と
      7 返事をする	て
      7 真似をやる	
      7 挨拶をする	で
      5 話をする	の
      5 喧嘩をする	で
      5 喧嘩をする	
      4 返事をする	
      4 質問をする	が て
      4 話をする	の に
      4 真似をする	の で
      4 挨拶をする	か と も
      3 返事をする	から ね と
      3 返事をする	さ と は に
      3 返事をする	の よ と
      3 話を聞く	の
      3 真似をする	の
      3 昼寝をする	が
      3 挨拶をする	から
      3 安心を得る	が
      2 遠慮をする	
      2 逆戻りをする	の
      2 辞儀をする	が
      2 質問をする	と
      2 質問をかける	か と が
      2 質問をかける	と は
      2 議論をする	て
      2 講釈をする	て ね
      2 講義をきく	
      2 話を聞く	
      2 話をする	
      2 行動をする	が
      2 落着を告げる	は と
      2 自炊をする	で に
      2 約束をする	て
      2 相談をする	と
      2 病気をする	
      2 深入りをする	
      2 活躍を試みる	て
      2 注意を惹く	の
      2 注意をする	で
      2 決心をする	と
      2 欠伸をする	て と て
      2 散歩をする	に
      2 散歩をする	
      2 放蕩をする	から が
      2 放蕩をする	
      2 挨拶をする	の は て と
      2 戦争をする	と
      2 復讐をとる	と の と は
      2 御無沙汰をする	
      2 平均を破る	の
      2 工夫をする	
      2 学問をする	
      2 喧嘩をする	て も
      2 問答をやる	の で
      2 問答をする	の
      2 告訴をかく	
      

## 48. 名詞から根へのパスの抽出

In [36]:
for chList in result[0:100]:
    for ch in chList:
        if ('名詞' not in [x.pos for x in ch.morphs]) or ch.dst==-1:continue
        res=[]
        tmpCh=ch
        res.append(''.join([x.surface for x in list(filter(lambda x: x.pos != '記号',ch.morphs))]))
        while tmpCh.dst!=-1:
            dstCh=chList[tmpCh.dst]
            res.append(''.join([x.surface for x in list(filter(lambda x: x.pos != '記号',dstCh.morphs))]))
            tmpCh=dstCh

        print(' -> '.join(res))

吾輩は -> 猫である
名前は -> 無い
どこで -> 生れたか -> つかぬ
見当が -> つかぬ
何でも -> 薄暗い -> 所で -> 泣いて -> 記憶している
所で -> 泣いて -> 記憶している
ニャーニャー -> 泣いて -> 記憶している
いた事だけは -> 記憶している
吾輩は -> 見た
ここで -> 始めて -> 人間という -> ものを -> 見た
人間という -> ものを -> 見た
ものを -> 見た
あとで -> 聞くと -> 種族であったそうだ
それは -> 種族であったそうだ
書生という -> 人間中で -> 種族であったそうだ
人間中で -> 種族であったそうだ
一番 -> 獰悪な -> 種族であったそうだ
獰悪な -> 種族であったそうだ
書生というのは -> 話である
我々を -> 捕えて -> 煮て -> 食うという -> 話である
当時は -> なかったから -> 思わなかった
何という -> 考も -> なかったから -> 思わなかった
考も -> なかったから -> 思わなかった
彼の -> 掌に -> 載せられて -> 持ち上げられた -> 時 -> フワフワした -> 感じが -> あったばかりである
掌に -> 載せられて -> 持ち上げられた -> 時 -> フワフワした -> 感じが -> あったばかりである
スーと -> 持ち上げられた -> 時 -> フワフワした -> 感じが -> あったばかりである
時 -> フワフワした -> 感じが -> あったばかりである
感じが -> あったばかりである
掌の -> 上で -> 落ちついて -> 見たのが -> 人間という -> ものの -> 見始であろう
上で -> 落ちついて -> 見たのが -> 人間という -> ものの -> 見始であろう
書生の -> 顔を -> 見たのが -> 人間という -> ものの -> 見始であろう
顔を -> 見たのが -> 人間という -> ものの -> 見始であろう
見たのが -> 人間という -> ものの -> 見始であろう
人間という -> ものの -> 見始であろう
ものの -> 見始であろう
時 -> ものだと -> 思った -> 感じが -> 残っている
妙な -> ものだと -> 思った -> 感じが -

## 49. 名詞間の係り受けパスの抽出

In [16]:
import itertools
# methodlize 48
def chLsToRoot(startCh):
    res=[]
    tmpCh=startCh
    res.append(tmpCh)
    while tmpCh.dst!=-1:
        dstCh=chList[tmpCh.dst]
        res.append(dstCh)
        tmpCh=dstCh
    return res

for chList in result[0:100]:
    pairs=list(itertools.combinations([idx for idx,ch in enumerate(chList) if '名詞' in [x.pos for x in ch.morphs]], 2))
    if len(pairs)==0:continue
    for p in pairs:
        i,j=p[0],p[1]
        chIds2Root=[chList.index(ch) for ch in chLsToRoot(chList[i])]
        if chList.index(chList[j]) in chIds2Root:
            mosLs=[ch.morphs for ch in chList[i:j+1]]
            midArr=[''.join([x.surface for x in list(filter(lambda x: x.pos != '記号',mos))]) for mos in mosLs[1:-1]]
            repI=['X' if m.pos=='名詞' else m.surface for m in mosLs[0]]
            print(' -> '.join([''.join(repI)]+midArr+['Y']))
        else:
            k=sorted(list(set([chList.index(ch) for ch in chLsToRoot(chList[j])]) & set(chIds2Root)))[0]
            kInJ=[chList.index(ch) for ch in chLsToRoot(chList[j])].index(sorted(list(set([chList.index(ch) for ch in chLsToRoot(chList[j])]) & set(chIds2Root)))[0])
            repI=['X' if m.pos=='名詞' else m.surface for m in chList[i].morphs]
            repJ=['Y' if m.pos=='名詞' else m.surface for m in chList[j].morphs]
            mosLs=[ch.morphs for ch in chLsToRoot(chList[j])]
            midTxt=[''.join([x.surface for x in list(filter(lambda x: x.pos != '記号',mos))]) for mos in mosLs[1:kInJ]]
            print(''.join(repI),
                  ' -> '.join([''.join(repJ)]+midTxt),
                  ''.join([x.surface for x in list(filter(lambda x: x.pos != '記号',chList[k].morphs))]),
                  sep=' | '
                 )

Xは -> Y
　Xで | Yが | つかぬ
Xでも -> 薄暗い -> じめじめした -> Y
Xでも | Y | 泣いて
Xでも | Yだけは | 記憶している
Xでも -> 薄暗い -> じめじめした -> 所で -> ニャーニャー -> 泣いて -> いた事だけは -> Y
Xで | Y | 泣いて
Xで | Yだけは | 記憶している
Xで -> ニャーニャー -> 泣いて -> いた事だけは -> Y
X | Yだけは | 記憶している
X -> 泣いて -> いた事だけは -> Y
Xだけは -> Y
Xは | Yで -> 始めて -> 人間という -> ものを | 見た
Xは | Yという -> ものを | 見た
Xは | Yを | 見た
Xで -> 始めて -> Y
Xで -> 始めて -> 人間という -> Y
Xという -> Y
Xで | Yは | 種族であったそうだ
Xで | Yという -> 人間中で | 種族であったそうだ
Xで | YYで | 種族であったそうだ
Xで | Y -> 獰悪な | 種族であったそうだ
Xで | Yな | 種族であったそうだ
Xで -> 聞くと -> それは -> 書生という -> 人間中で -> 一番 -> 獰悪な -> Y
Xは | Yという -> 人間中で | 種族であったそうだ
Xは | YYで | 種族であったそうだ
Xは | Y -> 獰悪な | 種族であったそうだ
Xは | Yな | 種族であったそうだ
Xは -> 書生という -> 人間中で -> 一番 -> 獰悪な -> Y
Xという -> Y
Xという | Y -> 獰悪な | 種族であったそうだ
Xという | Yな | 種族であったそうだ
Xという -> 人間中で -> 一番 -> 獰悪な -> Y
XXで | Y -> 獰悪な | 種族であったそうだ
XXで | Yな | 種族であったそうだ
XXで -> 一番 -> 獰悪な -> Y
X -> Y
X -> 獰悪な -> Y
Xな -> Y
XというXは | Yを -> 捕えて -> 煮て -> 食うという | 話である
XというXは -> 時々 -> 我々を -> 捕えて -> 煮て -> 食うという -> Y
Xを -> 捕えて -> 煮て -> 食うという -> Y
Xは | Yと